In [1]:
import os
from pathlib import Path

from logs.wandblogger import WandBLogger2D
from training.trainer import MRTrainer
from datasets.signals import ImageSignal#, make_mask
from networks.mrnet import MRFactory
from datasets.pyramids import create_MR_structure
import yaml
from yaml.loader import SafeLoader
import os

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "train-wb.ipynb"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
BASE_DIR = Path('.').absolute().parents[0]
IMAGE_PATH = BASE_DIR.joinpath('img')
MODEL_PATH = BASE_DIR.joinpath('models')

In [3]:
project_name = "dev-sandbox"
#-- hyperparameters in configs --#
config_file = '../configs/config_base_m_net.yml'
with open(config_file) as f:
    hyper = yaml.load(f, Loader=SafeLoader)
    if isinstance(hyper['batch_size'], str):
        hyper['batch_size'] = eval(hyper['batch_size'])
    print(hyper)
imgpath = os.path.join(IMAGE_PATH, hyper['image_name'])
maskpath = None
# maskpath = "/Users/hallpaz/Workspace/impa/mrimg/img/synthetic/mask_inverted.png" #make_mask(imgpath, hyper['mask_color'])

{'model': 'M', 'positive_freqs': False, 'in_features': 2, 'out_features': 1, 'hidden_layers': 1, 'hidden_features': [[512, 256], [48, 32], [80, 64], [192, 160], [256, 256], [512, 512]], 'bias': True, 'max_stages': 1, 'period': 4, 'pmode': 'reflect', 'domain': [-1, 1], 'omega_0': [32, 6, 16, 32, 64, 128], 'hidden_omega_0': [30, 30, 30, 30, 30, 30, 30], 'superposition_w0': False, 'sampling_scheme': 'reflect', 'decimation': True, 'filter': 'gauss', 'attributes': ['d0', 'd1'], 'loss_function': 'hermite', 'loss_weights': {'d0': 1, 'd1': 0}, 'opt_method': 'Adam', 'lr': 0.0001, 'loss_tol': 1e-11, 'diff_tol': 1e-07, 'max_epochs_per_stage': [800, 902, 802, 602, 402, 202, 102], 'batch_size': 65536, 'image_name': 'pexels_textures/pic7.png', 'width': 256, 'height': 256, 'channels': 1, 'YCbCr': False, 'device': 'cuda', 'eval_device': 'cpu', 'save_format': 'general', 'visualize_grad': True, 'extrapolate': [-2, 2], 'zoom': [2, 4]}


In [4]:
base_signal = ImageSignal.init_fromfile(
                    imgpath,
                    domain=hyper['domain'],
                    channels=hyper['channels'],
                    sampling_scheme=hyper['sampling_scheme'],
                    width=hyper['width'], height=hyper['height'],
                    attributes=hyper['attributes'],
                    batch_size=hyper['batch_size'],
                    YCbCr=hyper.get('YCbCr', False))

train_dataset = create_MR_structure(base_signal, 
                                       hyper['max_stages'], 
                                       hyper['filter'], 
                                       hyper['decimation'],
                                       hyper['pmode'])
test_dataset = create_MR_structure(base_signal, 
                                      hyper['max_stages'], 
                                      hyper['filter'], 
                                      False,
                                      hyper['pmode'])

In [5]:
hyper['device']

'cuda'

In [6]:
img_name = os.path.basename(hyper['image_name'])
mrmodel = MRFactory.from_dict(hyper)
print("Model: ", type(mrmodel))
wandblogger = WandBLogger2D(project_name,
                            f"{hyper['model']}{hyper['filter'][0].upper()}{img_name[0:5]}",
                            hyper,
                            BASE_DIR)
mrtrainer = MRTrainer.init_from_dict(mrmodel, 
                                     train_dataset, 
                                     test_dataset, 
                                     wandblogger, 
                                     hyper)
mrtrainer.train(hyper['device'])

Model:  <class 'networks.mrnet.MNet'>


wandb: Currently logged in as: hallpaz (siren-song). Use `wandb login --relogin` to force relogin


DATA SIZE torch.Size([1, 256, 256])
[Logger] All inference done in 3.9190919399261475s on cpu
File  MGpic7._1-1_w32F_hf512256_MEp800_hl1_r256_pr4.pth


D0 loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
D1 loss,▁▂▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████
Total loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
D0 loss,0.00027
D1 loss,75.40367
Total loss,0.00027


Total model parameters =  133120
Training finished after 800 epochs


: 